In [ ]:
import polars as pl
import gtfs_delay_analysis as da 

In [ ]:
ddelay = pl.read_parquet('all_ddelay.parquet').with_columns(pl.col('trip_id').cast(pl.Int64))
trips = da.trips.load_trips_without_shapes_df()
shapes = da.trips.load_parsed_shapes_df()

In [ ]:
with_shape_id = ddelay.join(trips.select('trip_id', 'shape_id'), on='trip_id')

In [ ]:
shapes_with_trip_id = shapes.join(trips.select('trip_id', 'shape_id').unique(), on='shape_id').drop('line_length')

In [ ]:
shapes_with_trip_id.select(
    'shape_id',
    'trip_id',
    pl.col('geometry_line').list.n_unique()
)['geometry_line'].sum()

In [ ]:
(
shapes_with_trip_id
.filter(pl.col('shape_id') == "508-1-East")
.explode('geometry_line')
.with_row_index()
.unique(['trip_id', 'geometry_line'], keep='first', maintain_order=True)
)

In [ ]:
selected_shape = pl.col('shape_id').eq("508-1-East")
pl.Config.set_tbl_rows(62)
exploded = (
shapes_with_trip_id
.filter(selected_shape)
.explode('geometry_line')
.unique(['trip_id', 'geometry_line'], keep='first', maintain_order=True)
)

In [ ]:
exploded.join(
    with_shape_id.filter(selected_shape),
    left_on=[pl.col('geometry_line').struct.field('lon'),pl.col('geometry_line').struct.field('lat'), 'trip_id','shape_id'],
    right_on=['a_lon', 'a_lat', 'trip_id', 'shape_id'],
    how='left'
)

In [ ]:
exploded.with_columns(
    pl.col('geometry_line').struct.field('lon'),
    pl.col('geometry_line').struct.field('lat'),
).drop('geometry_line').filter(pl.col('trip_id').eq(25536564))
with_shape_id.filter(selected_shape).select('a', 'a_lon', 'b_lon').unique()